# Exercise Classification Model Training

This notebook demonstrates the training of exercise classification models using the extracted features. We'll train models for various exercise attributes including:

- Target muscle groups
- Required equipment
- Movement patterns
- Intensity/experience levels
- Exercise types
- Quality of movement
- Risk assessment

In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.pytorch
from tqdm.notebook import tqdm

# Set paths
MODEL_DIR = "models"
RESULTS_DIR = "results"
DATA_DIR = "data/processed"

# Create directories
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 1. Load and Prepare Data

First, let's load the processed features and prepare them for training.

In [ ]:
# Load processed features
features_path = f"{DATA_DIR}/combined_features.csv"
if os.path.exists(features_path):
    features_df = pd.read_csv(features_path)
    print(f"Loaded {len(features_df)} exercises with {len(features_df.columns)} features")
else:
    print("Processed features not found. Please run the feature engineering notebook first.")
    features_df = None

# Define feature columns
if features_df is not None:
    feature_columns = {
        'text': [col for col in features_df.columns if col.startswith(('tfidf_', 'text_pca_'))],
        'pose': [col for col in features_df.columns if col.startswith(('landmark_', 'pose_pca_'))],
        'targets': [
            'bodypart', 'target', 'equipment', 'movement_pattern',
            'intensity_level', 'exercise_type', 'movement_quality',
            'risk_assessment'
        ]
    }
    
    print("\nFeature columns:")
    for key, cols in feature_columns.items():
        print(f"{key}: {len(cols)} columns")

## 2. Define Model Architecture

Let's define our multimodal neural network architecture.

In [ ]:
class ExerciseDataset(Dataset):
    """PyTorch Dataset for exercise data"""
    def __init__(self, text_features, pose_features, labels):
        self.text_features = torch.tensor(text_features, dtype=torch.float32)
        self.pose_features = torch.tensor(pose_features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.text_features[idx], self.pose_features[idx], self.labels[idx]

class MultimodalExerciseClassifier(nn.Module):
    """Neural network for classifying exercises using multimodal features"""
    def __init__(self, text_input_dim, pose_input_dim, num_classes, hidden_dim=128):
        super().__init__()
        
        # Text processing branch
        self.text_layers = nn.Sequential(
            nn.Linear(text_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Pose processing branch
        self.pose_layers = nn.Sequential(
            nn.Linear(pose_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Combined layers
        self.combined_layers = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, text_features, pose_features):
        text_output = self.text_layers(text_features)
        pose_output = self.pose_layers(pose_features)
        combined = torch.cat((text_output, pose_output), dim=1)
        return self.combined_layers(combined)

## 3. Define Training Functions

Let's create the functions needed for training and evaluation.

In [ ]:
def prepare_data(df, target_col, text_cols, pose_cols, test_size=0.2, val_size=0.1):
    """Prepare data for training"""
    # Extract features
    text_features = df[text_cols].fillna(0).values if text_cols else np.zeros((len(df), 1))
    pose_features = df[pose_cols].fillna(0).values if pose_cols else np.zeros((len(df), 1))
    
    # Encode target
    encoder = LabelEncoder()
    labels = encoder.fit_transform(df[target_col])
    
    # Split data
    X_train_text, X_test_text, X_train_pose, X_test_pose, y_train, y_test = train_test_split(
        text_features, pose_features, labels, test_size=test_size, random_state=42, stratify=labels
    )
    
    X_train_text, X_val_text, X_train_pose, X_val_pose, y_train, y_val = train_test_split(
        X_train_text, X_train_pose, y_train, test_size=val_size/(1-test_size), random_state=42, stratify=y_train
    )
    
    # Create datasets
    train_dataset = ExerciseDataset(X_train_text, X_train_pose, y_train)
    val_dataset = ExerciseDataset(X_val_text, X_val_pose, y_val)
    test_dataset = ExerciseDataset(X_test_text, X_test_pose, y_test)
    
    return {
        'train_dataset': train_dataset,
        'val_dataset': val_dataset,
        'test_dataset': test_dataset,
        'encoder': encoder
    }

def train_model(data, target_col, batch_size=32, learning_rate=0.001, epochs=30, patience=5):
    """Train a model for a specific target"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create data loaders
    train_loader = DataLoader(data['train_dataset'], batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(data['val_dataset'], batch_size=batch_size)
    
    # Create model
    text_dim = data['train_dataset'].text_features.shape[1]
    pose_dim = data['train_dataset'].pose_features.shape[1]
    num_classes = len(data['encoder'].classes_)
    
    model = MultimodalExerciseClassifier(text_dim, pose_dim, num_classes).to(device)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for text, pose, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} (Train)'):
            text, pose, labels = text.to(device), pose.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(text, pose)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for text, pose, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} (Val)'):
                text, pose, labels = text.to(device), pose.to(device), labels.to(device)
                
                outputs = model(text, pose)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'target_col': target_col,
                'feature_names': {
                    'text': text_cols,
                    'pose': pose_cols
                }
            }, f"{MODEL_DIR}/{target_col}_model.pth")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered after {epoch+1} epochs')
                break
    
    return {
        'model': model,
        'history': history,
        'target_col': target_col
    }

def evaluate_model(model, test_dataset, encoder, target_col):
    """Evaluate a trained model"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    test_loader = DataLoader(test_dataset, batch_size=32)
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for text, pose, labels in test_loader:
            text, pose = text.to(device), pose.to(device)
            outputs = model(text, pose)
            _, predicted = torch.max(outputs.data, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=encoder.classes_)
    
    return {
        'accuracy': accuracy,
        'classification_report': report
    }

## 4. Train Models

Now, let's train models for each target variable.

In [ ]:
# Select target variables to train models for
if features_df is not None and 'feature_columns' in locals():
    # You can select specific targets here or use all available ones
    target_variables = feature_columns['targets']
    
    print(f"Training models for {len(target_variables)} target variables:")
    for target in target_variables:
        print(f"  - {target}")
    
    # Ask for confirmation
    confirm = input("\nProceed with training? (y/n): ")
    
    if confirm.lower() == 'y':
        # Dictionary to store training results
        training_results = {}
        
        # Train a model for each target variable
        for target in target_variables:
            print(f"\n{'='*80}\nTraining model for {target}\n{'='*80}")
            
            # Prepare data for this target
            data = prepare_data(
                features_df, 
                target, 
                feature_columns['text'], 
                feature_columns['pose'],
                test_size=0.2,
                val_size=0.1
            )
            
            # Train model
            result = train_model(
                data,
                target,
                batch_size=32,
                learning_rate=0.001,
                epochs=30,
                patience=5
            )
            
            # Evaluate model
            print(f"\nEvaluating model for {target}")
            eval_result = evaluate_model(
                result['model'],
                data['test_dataset'],
                data['encoder'],
                target
            )
            
            # Store results
            training_results[target] = {
                "training": result,
                "evaluation": eval_result
            }
        
        print("\nTraining complete! Results summary:")
        for target, result in training_results.items():
            print(f"\n{target}:")
            print(f"  Accuracy: {result['evaluation']['accuracy']:.4f}")
            if 'classification_report' in result['evaluation']:
                report = result['evaluation']['classification_report']
                if 'weighted avg' in report:
                    print(f"  Precision: {report['weighted avg']['precision']:.4f}")
                    print(f"  Recall: {report['weighted avg']['recall']:.4f}")
                    print(f"  F1 Score: {report['weighted avg']['f1-score']:.4f}")
    else:
        print("Training cancelled.")

## 5. Visualize Model Results

Let's visualize the results of our trained models.

In [ ]:
# Visualize model results
if 'training_results' in locals() and training_results:
    # Create figure for accuracy comparison
    plt.figure(figsize=(12, 6))
    
    # Extract accuracies
    targets = list(training_results.keys())
    accuracies = [result['evaluation']['accuracy'] for result in training_results.values()]
    
    # Sort by accuracy
    sorted_indices = np.argsort(accuracies)
    sorted_targets = [targets[i] for i in sorted_indices]
    sorted_accuracies = [accuracies[i] for i in sorted_indices]
    
    # Plot accuracies
    ax = sns.barplot(x=sorted_accuracies, y=sorted_targets)
    
    # Add accuracy values
    for i, acc in enumerate(sorted_accuracies):
        ax.text(acc + 0.02, i, f"{acc:.4f}", va='center')
    
    plt.title("Model Accuracy Comparison")
    plt.xlabel("Accuracy")
    plt.ylabel("Target Variable")
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/model_accuracy_comparison.png")
    plt.show()

## 6. Make Predictions

Let's show how to use the trained models to make predictions on new exercises.

In [ ]:
def load_model(target_col, model_dir=MODEL_DIR):
    """Load a trained model"""
    model_path = f"{model_dir}/{target_col}_model.pth"
    if not os.path.exists(model_path):
        print(f"Model for {target_col} not found")
        return None
    
    checkpoint = torch.load(model_path)
    
    # Create model with same architecture
    text_dim = len(checkpoint['feature_names']['text'])
    pose_dim = len(checkpoint['feature_names']['pose'])
    num_classes = len(checkpoint['target_col'].split(','))
    
    model = MultimodalExerciseClassifier(text_dim, pose_dim, num_classes)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return {
        'model': model,
        'metadata': checkpoint
    }

def predict(model, text_features, pose_features, text_scaler, pose_scaler, encoder):
    """Make predictions using a trained model"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    # Scale features
    text_features = text_scaler.transform(text_features)
    pose_features = pose_scaler.transform(pose_features)
    
    # Convert to tensors
    text_tensor = torch.tensor(text_features, dtype=torch.float32).to(device)
    pose_tensor = torch.tensor(pose_features, dtype=torch.float32).to(device)
    
    # Make predictions
    with torch.no_grad():
        outputs = model(text_tensor, pose_tensor)
        _, predicted = torch.max(outputs, 1)
    
    # Convert to numpy and decode
    predicted = predicted.cpu().numpy()
    predicted = encoder.inverse_transform(predicted)
    
    return predicted

# Example: Load a model and make predictions
if 'feature_columns' in locals():
    target_to_predict = feature_columns['targets'][0]  # Use first target as example
    
    print(f"Loading model for {target_to_predict}")
    loaded_model = load_model(target_to_predict)
    
    if loaded_model:
        print(f"Model loaded successfully")
        print(f"Model metadata: {loaded_model['metadata']}")
        
        # Select some examples to predict
        num_examples = 5
        examples = features_df.sample(num_examples)
        
        # Extract features
        text_features = examples[feature_columns['text']].fillna(0).values
        pose_features = examples[feature_columns['pose']].fillna(0).values
        
        # Make predictions
        predictions = predict(
            loaded_model['model'],
            text_features,
            pose_features,
            loaded_model['metadata']['text_scaler'],
            loaded_model['metadata']['pose_scaler'],
            loaded_model['metadata']['encoder']
        )
        
        # Display results
        print(f"\nPredictions for {target_to_predict}:")
        for i, (_, example) in enumerate(examples.iterrows()):
            print(f"\nExample {i+1}:")
            print(f"  Name: {example.get('name', 'Unknown')}")
            print(f"  Body Part: {example.get('bodypart', 'Unknown')}")
            print(f"  Actual {target_to_predict}: {example.get(target_to_predict, 'Unknown')}")
            print(f"  Predicted {target_to_predict}: {predictions[i]}")